### CNN for predicting crop yield using preprocessed arrays
Author: Vivek Jayaswal \
Date: 20-Dec-2020

In [1]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

In [2]:
# Load custom functions
%run Zindi_CGIAR_Functions.ipynb

In [3]:
in_dir = 'C:/Work/Vivek_Work/Challenge_Data_Sets/Zindi/CGIAR/{}'
trng_dir = 'C:/Work/Vivek_Work/Challenge_Data_Sets/Zindi/CGIAR/image_arrays_train/{}_Rev.npy'
test_dir = 'C:/Work/Vivek_Work/Challenge_Data_Sets/Zindi/CGIAR/image_arrays_test/{}_Rev.npy'

trng_data_annot = pd.read_csv(in_dir.format('Train.csv'))

In [4]:
trng_data_annot.head()

,Field_ID,Year,Quality,Yield
0,MH2O0YH,2019,3,3.686
1,O9TURWL,2019,2,5.657
2,35AFSDD,2019,3,3.082
3,PM05EG9,2019,2,2.707
4,V7PZBCG,2019,2,2.679


In [5]:
# Remove poor quality images (quality = 1)
trng_data_annot = trng_data_annot[trng_data_annot['Quality'] != 1]
np.unique(trng_data_annot['Quality'], return_counts=True)

(array([2, 3], dtype=int64), array([1231, 1321], dtype=int64))

In [6]:
# Load all preprocessed 3-d arrays channels x bins x time into 4-d arrays
field_names = trng_data_annot['Field_ID']
num_fields = len(field_names)
field_arrays = np.zeros((num_fields, 27, 20, 12))

for current_idx in range(num_fields):
    if current_idx%500 == 0:
        print('Counter = ', current_idx)
    temp_array = np.load(trng_dir.format(field_names.values[current_idx]))
    field_arrays[current_idx] = temp_array

Counter =  0
Counter =  500
Counter =  1000
Counter =  1500
Counter =  2000
Counter =  2500


In [7]:
field_arrays[0, 0, :, :].sum()

1.0

### Option 1: Dataset and DataLoader Generation
Use of custom Dataset Class

In [8]:
# Generate custom Dataset object for use with DataLoader for batch creation
# Need to pass NumPy Arrays and not Pandas objects
zindi_dataset = ZindiDataset(field_arrays, trng_data_annot['Yield'].values)

In [9]:
# Input array and its value
print('Array dimensions = ', zindi_dataset[0][0].shape)
print('Yield = ', zindi_dataset[0][1])

Array dimensions =  (27, 20, 12)
Yield =  3.686


In [10]:
# Generate batches of size 32
zindi_dataloader = torch.utils.data.DataLoader(zindi_dataset, batch_size=32, shuffle=False)
len(zindi_dataloader)

80

In [11]:
# Number of batches of size 32
2552/32

79.75

In [12]:
# Display the batch details
temp = iter(zindi_dataloader)

In [13]:
next(temp)

[tensor([[[[8.1229e-02, 8.8459e-02, 7.8600e-02,  ..., 9.2074e-02,
            9.2074e-02, 8.2982e-02],
           [0.0000e+00, 3.6151e-03, 1.3474e-02,  ..., 0.0000e+00,
            0.0000e+00, 5.1487e-03],
           [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 1.9718e-03],
           ...,
           [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00]],
 
          [[4.3817e-02, 2.4180e-02, 1.8230e-02,  ..., 6.6861e-02,
            6.0857e-02, 7.1838e-02],
           [4.6630e-02, 6.0965e-02, 6.7348e-02,  ..., 2.3585e-02,
            2.9644e-02, 1.7960e-02],
           [2.7047e-04, 5.6800e-03, 5.2472e-03,  ..., 3.7866e-04,
            4.3276e-04, 8.1142e-04],
           ...,
           [0.0000e+00, 0.0000e+00, 0.

### Option 2: Dataset and DataLoader Generation
Use of TensorDataset

In [14]:
# An alternate approach to generating Dataset
rev_zindi_dataset = TensorDataset(torch.FloatTensor(field_arrays),torch.FloatTensor(trng_data_annot['Yield']))

In [15]:
rev_zindi_dataset[0]

(tensor([[[0.0812, 0.0885, 0.0786,  ..., 0.0921, 0.0921, 0.0830],
          [0.0000, 0.0036, 0.0135,  ..., 0.0000, 0.0000, 0.0051],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0020],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
 
         [[0.0438, 0.0242, 0.0182,  ..., 0.0669, 0.0609, 0.0718],
          [0.0466, 0.0610, 0.0673,  ..., 0.0236, 0.0296, 0.0180],
          [0.0003, 0.0057, 0.0052,  ..., 0.0004, 0.0004, 0.0008],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
 
         [[0.0070, 0.0027, 0.0012,  ..., 0.0014, 0.0012, 0.0486],
          [0.0490, 0.0284, 0.0287,  ..., 0.0360, 0.0482, 0.0292],
          [0.0346, 0.0545, 0.0557,  ...,

In [16]:
# Generate batches of size 32
zindi_dataloader = torch.utils.data.DataLoader(rev_zindi_dataset, batch_size=32, shuffle=False)
len(zindi_dataloader)

80

In [17]:
temp = iter(zindi_dataloader)

In [18]:
next(temp)

[tensor([[[[8.1229e-02, 8.8459e-02, 7.8600e-02,  ..., 9.2074e-02,
            9.2074e-02, 8.2982e-02],
           [0.0000e+00, 3.6151e-03, 1.3474e-02,  ..., 0.0000e+00,
            0.0000e+00, 5.1487e-03],
           [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 1.9718e-03],
           ...,
           [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00]],
 
          [[4.3817e-02, 2.4180e-02, 1.8230e-02,  ..., 6.6861e-02,
            6.0857e-02, 7.1838e-02],
           [4.6630e-02, 6.0965e-02, 6.7348e-02,  ..., 2.3585e-02,
            2.9644e-02, 1.7960e-02],
           [2.7047e-04, 5.6800e-03, 5.2472e-03,  ..., 3.7866e-04,
            4.3276e-04, 8.1142e-04],
           ...,
           [0.0000e+00, 0.0000e+00, 0.

In [19]:
# Data for first batch using 'Method 1'
# tensor([3.6860, 5.6570, 3.0820, 2.7070, 2.6790, 4.3170, 2.9990, 4.1990, 3.5050,
#          5.0390, 3.1720, 2.3530, 2.9570, 3.2690, 3.1170, 2.7900, 4.8590, 2.5680,
#          0.9860, 3.4980, 2.6380, 4.4560, 4.1650, 4.0190, 3.5470, 2.4430, 4.3150,
#          2.2360, 3.4190, 3.4710, 4.0810, 0.4580], dtype=torch.float64)]

### Generate training and validation sets in batches of size 32

In [20]:
rng = np.random.default_rng(100)

num_obs = len(field_arrays)
trng_set_idx = rng.choice(num_obs, (np.ceil(num_obs * 2/3)).astype(np.int16), replace=False)
valid_set_idx = list(set(range(num_obs)) - set(trng_set_idx))

In [21]:
union_set = set(valid_set_idx) | set(trng_set_idx)
len(union_set)

2552

In [22]:
print(len(trng_set_idx))
1702/32

1702


53.1875

In [23]:
zz = trng_data_annot['Yield']
print(zz[0:6])
zz.iloc[[0, 2, 5]]

0    3.686
1    5.657
2    3.082
3    2.707
4    2.679
5    4.317
Name: Yield, dtype: float64


0    3.686
2    3.082
5    4.317
Name: Yield, dtype: float64

In [24]:
# Dummy cell for executing all steps below it

In [25]:
# Randomize the generation of training and validation set
num_obs = len(field_arrays)
trng_set_idx = np.random.choice(num_obs, (np.ceil(num_obs * 2/3)).astype(np.int16), replace=False)
valid_set_idx = list(set(range(num_obs)) - set(trng_set_idx))

zindi_dataset_trng = TensorDataset(torch.FloatTensor(field_arrays[trng_set_idx])
                                   ,torch.FloatTensor(trng_data_annot['Yield'].values[trng_set_idx]))

# Use shuffle = True to change the order of observations across epochs
zindi_dataloader_trng = torch.utils.data.DataLoader(zindi_dataset_trng, batch_size=32, shuffle=True)
len(zindi_dataloader_trng)

54

In [26]:
zindi_dataset_valid = TensorDataset(torch.FloatTensor(field_arrays[valid_set_idx])
                                   ,torch.FloatTensor(trng_data_annot['Yield'].values[valid_set_idx]))

zindi_dataloader_valid = torch.utils.data.DataLoader(zindi_dataset_valid, batch_size=32, shuffle=True)
len(zindi_dataloader_valid)

27

### Create a CNN for predicting the yield per acre

In [27]:
# Number of variables to be flattened
14*6*3

252

In [28]:
# Set seed for initialization of model parameters
torch.manual_seed(10)

In [29]:
class ZindiCnn(torch.nn.Module):
    def __init__(self, num_channels):
        super().__init__()
        self.layer_1 = torch.nn.Conv2d(in_channels = num_channels, out_channels = 14, kernel_size = (1, 3))
        self.pool_1 = torch.nn.AvgPool2d((3, 1), stride=1)
#         self.layer_1 = torch.nn.Conv2d(in_channels = num_channels, out_channels = 14, kernel_size = (3, 3))
        self.layer_2 = torch.nn.Conv2d(in_channels = 14, out_channels = 7, kernel_size = 3)
        self.layer_3 = torch.nn.Conv2d(in_channels = 7, out_channels = 3, kernel_size = 3)
        self.layer_4 = torch.nn.Linear(252, 3)
        self.layer_5 = torch.nn.Linear(3, 1)
        
    def forward(self, x):
        # define forward pass
        x = self.layer_1(x)
        x = F.relu(self.pool_1(x))
        x = self.layer_2(x)
        x = self.layer_3(x)
#         print(x.size())
        x = x.view(-1, 252)
        x = F.relu(self.layer_4(x))
        x = self.layer_5(x)
        return x

In [30]:
# temp = iter(zindi_dataloader_trng)
# single_batch = next(temp)

In [31]:
# single_batch[0].size()

In [32]:
# single_batch[1].size()

In [33]:
# zindi_model = ZindiCnn(27)
# zindi_optimizer = torch.optim.Adam(zindi_model.parameters(), lr=0.01)
# zindi_loss_function = torch.nn.MSELoss()

In [34]:
# Train the model & evaluate generalization error
num_epochs = 101
# zindi_model.train()

zindi_model = ZindiCnn(27)
zindi_optimizer = torch.optim.Adam(zindi_model.parameters(), lr=0.001)
zindi_loss_function = torch.nn.MSELoss()

for epoch_idx in range(num_epochs):
    
    if epoch_idx%100 == 0:
        print(epoch_idx)
        test_loss = 0
        
        # Use model in evaluation mode
        zindi_model.eval()
        with torch.no_grad():
            for valid_batch in zindi_dataloader_valid:
                current_fit = zindi_model(valid_batch[0])
                test_loss += zindi_loss_function(current_fit, torch.unsqueeze(valid_batch[1], 1)).item()

        print('MSE on validation set = ', test_loss/len(zindi_dataloader_valid))
    
    zindi_model.train()
    for single_batch in zindi_dataloader_trng:
        current_fit = zindi_model(single_batch[0])  # 1
        current_loss = zindi_loss_function(current_fit, torch.unsqueeze(single_batch[1], 1))  # 2
        zindi_optimizer.zero_grad()  # 3
        current_loss.backward()  # 4
        zindi_optimizer.step()  # 5

0
MSE on validation set =  14.30005896532977
100
MSE on validation set =  2.4483317092612937


In [35]:
# print(current_fit.size())
# torch.unsqueeze(single_batch[1], 1).size()

In [36]:
single_batch[1]

tensor([4.6000, 4.5012, 1.4020, 1.1000, 3.1688, 5.8630])

In [37]:
zindi_model.eval()
test_loss = 0

with torch.no_grad():
    for valid_batch in zindi_dataloader_valid:
        current_fit = zindi_model(valid_batch[0])
        test_loss += zindi_loss_function(current_fit, torch.unsqueeze(valid_batch[1], 1)).item()

print('MSE on validation set = ', test_loss/len(zindi_dataloader_valid))

MSE on validation set =  2.527046706941393


In [38]:
zindi_model.layer_5.bias

Parameter containing:
tensor([-0.0650], requires_grad=True)

In [39]:
# To be executed for second or later runs of the MSE estimation code

# print(np.all(zindi_model.layer_1.weight.detach().numpy() == layer_1_weight))
# print(np.all(zindi_model.layer_1.bias.detach().numpy() == layer_1_bias))

# print(np.all(zindi_model.layer_2.weight.detach().numpy() == layer_2_weight))
# print(np.all(zindi_model.layer_2.bias.detach().numpy() == layer_2_bias))

# print(np.all(zindi_model.layer_3.weight.detach().numpy() == layer_3_weight))
# print(np.all(zindi_model.layer_3.bias.detach().numpy() == layer_3_bias))

# print(np.all(zindi_model.layer_4.weight.detach().numpy() == layer_4_weight))
# print(np.all(zindi_model.layer_4.bias.detach().numpy() == layer_4_bias))

# print(np.all(zindi_model.layer_5.weight.detach().numpy() == layer_5_weight))
# print(np.all(zindi_model.layer_5.bias.detach().numpy() == layer_5_bias))

### Results after first iteration on Validation Set

In [40]:
# Test the model using validation set
np.sqrt(test_loss/len(zindi_dataloader_valid))

1.5896687412607047

In [41]:
zindi_model.layer_5.weight

Parameter containing:
tensor([[0.1473, 0.3081, 0.3614]], requires_grad=True)

In [42]:
zindi_model.layer_5.bias

Parameter containing:
tensor([-0.0650], requires_grad=True)

In [43]:
layer_1_weight = zindi_model.layer_1.weight.detach().numpy()
layer_1_bias = zindi_model.layer_1.bias.detach().numpy()
layer_1_weight.shape

(14, 27, 1, 3)

In [44]:
layer_2_weight = zindi_model.layer_2.weight.detach().numpy()
layer_2_bias = zindi_model.layer_2.bias.detach().numpy()

layer_3_weight = zindi_model.layer_3.weight.detach().numpy()
layer_3_bias = zindi_model.layer_3.bias.detach().numpy()

layer_4_weight = zindi_model.layer_4.weight.detach().numpy()
layer_4_bias = zindi_model.layer_4.bias.detach().numpy()

layer_5_weight = zindi_model.layer_5.weight.detach().numpy()
layer_5_bias = zindi_model.layer_5.bias.detach().numpy()

### Predict the output for the test set

In [45]:
# Read the SampleSubmission.csv file
test_annot = pd.read_csv(in_dir.format('SampleSubmission.csv'))
test_annot.head()

,Field_ID,Yield
0,E9UZCEA,0
1,1WGGS1Q,0
2,EG2KXE2,0
3,HC3GQXF,0
4,7AK6GFK,0


In [46]:
test_sample_1 = torch.FloatTensor(np.load(test_dir.format(test_annot['Field_ID'].values[0])))
test_sample_2 = torch.FloatTensor(np.load(test_dir.format(test_annot['Field_ID'].values[1])))
test_batch = torch.stack((test_sample_1, test_sample_2), dim=0)
test_batch.shape

torch.Size([2, 27, 20, 12])

In [47]:
zindi_model.eval()

with torch.no_grad():
    current_fit = zindi_model(test_batch)

In [48]:
current_fit.numpy().flatten().tolist()

[3.3882675170898438, 3.4651482105255127]

In [49]:
# Read all test images
# Load all preprocessed 3-d arrays channels x bins x time into 4-d arrays
field_names = test_annot['Field_ID']
num_fields = len(field_names)
test_field_arrays = np.zeros((num_fields, 27, 20, 12))

for current_idx in range(num_fields):
    if current_idx%500 == 0:
        print('Counter = ', current_idx)
    temp_array = np.load(test_dir.format(field_names.values[current_idx]))
    test_field_arrays[current_idx] = temp_array
    
test_field_arrays.shape    

Counter =  0
Counter =  500
Counter =  1000


(1055, 27, 20, 12)

In [50]:
num_test_samples = test_field_arrays.shape[0]
predicted_yield = []

In [51]:
start_idx = list(range(0, num_test_samples, 32))
start_idx

[0,
 32,
 64,
 96,
 128,
 160,
 192,
 224,
 256,
 288,
 320,
 352,
 384,
 416,
 448,
 480,
 512,
 544,
 576,
 608,
 640,
 672,
 704,
 736,
 768,
 800,
 832,
 864,
 896,
 928,
 960,
 992,
 1024]

In [52]:
zindi_model.eval()

with torch.no_grad():
    for temp_idx in start_idx:
        # Obtain the start and end indices
        first_idx = 0 + temp_idx
        last_idx = min(num_test_samples, 32 + temp_idx)
        print('Index range from ', first_idx, ' to ', last_idx)
        
        current_indexes = list(range(first_idx, last_idx))
        current_fit = zindi_model(torch.FloatTensor(test_field_arrays[current_indexes]))
        predicted_yield = predicted_yield + current_fit.numpy().flatten().tolist()

Index range from  0  to  32
Index range from  32  to  64
Index range from  64  to  96
Index range from  96  to  128
Index range from  128  to  160
Index range from  160  to  192
Index range from  192  to  224
Index range from  224  to  256
Index range from  256  to  288
Index range from  288  to  320
Index range from  320  to  352
Index range from  352  to  384
Index range from  384  to  416
Index range from  416  to  448
Index range from  448  to  480
Index range from  480  to  512
Index range from  512  to  544
Index range from  544  to  576
Index range from  576  to  608
Index range from  608  to  640
Index range from  640  to  672
Index range from  672  to  704
Index range from  704  to  736
Index range from  736  to  768
Index range from  768  to  800
Index range from  800  to  832
Index range from  832  to  864
Index range from  864  to  896
Index range from  896  to  928
Index range from  928  to  960
Index range from  960  to  992
Index range from  992  to  1024
Index range fro

In [53]:
len(predicted_yield)

1055

In [54]:
np.array(predicted_yield).min()

1.8711638450622559

In [55]:
np.array(predicted_yield).max()

6.084647178649902

In [56]:
np.percentile(np.array(predicted_yield), [0, 25, 50, 75, 100])

array([1.87116385, 2.56673074, 2.99811721, 3.84472454, 6.08464718])

In [57]:
# Save the actuals yields in a new file
test_annot['Inferred_Yield'] = predicted_yield
test_annot.head()

,Field_ID,Yield,Inferred_Yield
0,E9UZCEA,0,3.388268
1,1WGGS1Q,0,3.465148
2,EG2KXE2,0,3.644513
3,HC3GQXF,0,3.547567
4,7AK6GFK,0,3.359572


In [58]:
test_annot.to_csv('VJ_Submission.csv', columns = ['Field_ID', 'Inferred_Yield'], header = ['Field_ID', 'Yield']
                 , index = False, )